In [1]:
# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


In [2]:
# general imports
from shutil import rmtree
import os
import time

In [3]:
# data science imports
import pandas as pd
import numpy as np
import csv

In [4]:
import cxr_dataset as CXR
import eval_model as E

In [5]:
use_gpu = torch.cuda.is_available()
gpu_count = torch.cuda.device_count()
print("Números de GPUs ativas:" + str(gpu_count))

Números de GPUs ativas:1


In [6]:
def checkpoint(model, best_loss, epoch, LR):
    """
    Salvando o checkpoint do modelo

    Args:
        model: modelo a ser salvo
        best_loss: melhor loss obtido até o momento
        epoch: número da epoch atual
        LR: learning rate atual
    Returns:
        None
    """

    print('Salvando checkpoint...')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, 'results/checkpoint')

In [7]:
from tqdm import tqdm

def train_model(
        model,
        criterion,
        optimizer,
        LR,
        num_epochs,
        dataloaders,
        dataset_sizes,
        weight_decay):
    """
    Ajusta um modelo torchvision para dados CXR da NIH.

    Args:
        model: modelo torchvision a ser ajustado (densenet-121 neste caso)
        criterion: critério de perda (perda de entropia cruzada binária, BCELoss)
        optimizer: otimizador a ser usado no treinamento (SGD)
        LR: taxa de aprendizado
        num_epochs: continuar o treinamento até este número de epochs
        dataloaders: dataloaders de treinamento e validação do PyTorch
        dataset_sizes: comprimento dos datasets de treinamento e validação
        weight_decay: parâmetro de decaimento de peso que usamos no SGD com momentum
    Returns:
        model: modelo torchvision treinado
        best_epoch: epoch em que a melhor perda de validação do modelo foi obtida

    """
    since = time.time()

    start_epoch = 1
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1

    # iterar sobre as epochs
    for epoch in range(start_epoch, num_epochs + 1):
        print('epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # definir o modelo para o modo de treinamento ou avaliação com base em
        # se estamos no treinamento ou na validação; necessário para obter previsões corretas dada a batchnorm
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0

            i = 0
            total_done = 0
            # iterar sobre todos os dados no dataloader de treinamento/validação:
            for data in tqdm(dataloaders[phase]):
                i += 1
                inputs, labels, _ = data
                batch_size = inputs.shape[0]
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda()).float()
                outputs = model(inputs)

                # calcular o gradiente e atualizar os parâmetros na fase de treinamento
                optimizer.zero_grad()
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data * batch_size

            epoch_loss = running_loss / dataset_sizes[phase]

            if phase == 'train':
                last_train_loss = epoch_loss

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))

            # diminuir a taxa de aprendizado se não houver melhoria na perda de validação nesta epoch
            if phase == 'val' and epoch_loss > best_loss:
                print("diminuindo a taxa de aprendizado de " + str(LR) + " para " +
                      str(LR / 10) + " pois não estamos vendo melhoria na perda de validação")
                LR = LR / 10
                # criar um novo otimizador com uma taxa de aprendizado menor
                optimizer = optim.SGD(
                    filter(
                        lambda p: p.requires_grad,
                        model.parameters()),
                    lr=LR,
                    momentum=0.9,
                    weight_decay=weight_decay)
                print("criado novo otimizador com taxa de aprendizado " + str(LR))

            # salvar um checkpoint do modelo se tiver a melhor perda de validação até agora
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, LR)

            # registrar a perda de treinamento e validação em cada epoch
            if phase == 'val':
                with open("results/log_train", 'a') as logfile:
                    logwriter = csv.writer(logfile, delimiter=',')
                    if(epoch == 1):
                        logwriter.writerow(["epoch", "train_loss", "val_loss"])
                    logwriter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if(total_done % (100 * batch_size) == 0):
            print("completado " + str(total_done) + " até agora na epoch")

        # interromper se não houver melhoria na perda de validação em 3 epochs
        if ((epoch - best_epoch) >= 3):
            print("sem melhoria em 3 epochs, interrompendo")
            break

    time_elapsed = time.time() - since
    print('Treinamento completo em {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # carregar os melhores pesos do modelo para retornar
    checkpoint_best = torch.load('results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch

In [8]:
def train_cnn(PATH_TO_IMAGES, LR, WEIGHT_DECAY):
    """
    Treina um modelo torchvision com dados da NIH, dados hiperparâmetros de alto nível.

    Args:
        PATH_TO_IMAGES: caminho para as imagens da NIH
        LR: taxa de aprendizado
        WEIGHT_DECAY: parâmetro de decaimento de peso para SGD

    Returns:
        preds: previsões do modelo torchvision no conjunto de teste com a verdadeira para comparação
        aucs: AUCs para cada par de treino e teste

    """
    NUM_EPOCHS = 20
    BATCH_SIZE = 14

    try:
        rmtree('results/')
    except BaseException:
        pass  # o diretório ainda não existe, não há necessidade de limpá-lo
    os.makedirs("results/")

    # use a média e o desvio padrão do ImageNet para normalização
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    N_LABELS = 14  # estamos prevendo 14 rótulos

    # defina as transformações torchvision
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # porque o redimensionamento nem sempre dá 224 x 224, isso garante 224 x 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

    # crie dataloaders de treino/val
    transformed_datasets = {}
    transformed_datasets['train'] = CXR.CXRDataset(
        path_to_images=PATH_TO_IMAGES,
        fold='train',
        transform=data_transforms['train'])
    transformed_datasets['val'] = CXR.CXRDataset(
        path_to_images=PATH_TO_IMAGES,
        fold='val',
        transform=data_transforms['val'])

    dataloaders = {}
    dataloaders['train'] = torch.utils.data.DataLoader(
        transformed_datasets['train'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8)
    dataloaders['val'] = torch.utils.data.DataLoader(
        transformed_datasets['val'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8)
    
    
    if not use_gpu:
        raise ValueError("Erro, requer GPU")
    
    model = models.densenet121(weights='DEFAULT')
    num_ftrs = model.classifier.in_features
    # adicione a camada final com # de saídas na mesma dimensão dos rótulos com ativação sigmoidal
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, N_LABELS), nn.Sigmoid())

    # coloque o modelo na GPU
    model = model.cuda()

    # defina o critério, otimizador para treinamento
    criterion = nn.BCELoss()
    optimizer = optim.SGD(
        filter(
            lambda p: p.requires_grad,
            model.parameters()),
        lr=LR,
        momentum=0.9,
        weight_decay=WEIGHT_DECAY)
    dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val']}

    # treine o modelo
    model, best_epoch = train_model(model, criterion, optimizer, LR, num_epochs=NUM_EPOCHS,
                                    dataloaders=dataloaders, dataset_sizes=dataset_sizes, weight_decay=WEIGHT_DECAY)

    # obtenha previsões e AUCs no conjunto de teste
    preds, aucs = E.make_pred_multilabel(
        data_transforms, model, PATH_TO_IMAGES)

    return preds, aucs

In [9]:
PATH_TO_IMAGES = "./images/"
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.01
preds, aucs = train_cnn(PATH_TO_IMAGES, LEARNING_RATE, WEIGHT_DECAY)

epoch 1/20
----------


100%|██████████| 5605/5605 [08:51<00:00, 10.54it/s]


train epoch 1:loss 0.1623 with data size 78468


100%|██████████| 802/802 [00:47<00:00, 16.96it/s]


val epoch 1:loss 0.1530 with data size 11219
Salvando checkpoint...
epoch 2/20
----------


...

val epoch 10:loss 0.1471 with data size 11219
diminuindo a taxa de aprendizado de 1.0000000000000002e-06 para 1.0000000000000002e-07 pois não estamos vendo melhoria na perda de validação
criado novo otimizador com taxa de aprendizado 1.0000000000000002e-07
epoch 11/20
----------


100%|██████████| 5605/5605 [08:45<00:00, 10.66it/s]


train epoch 11:loss 0.1232 with data size 78468


100%|██████████| 802/802 [00:43<00:00, 18.55it/s]


val epoch 11:loss 0.1469 with data size 11219
diminuindo a taxa de aprendizado de 1.0000000000000002e-07 para 1.0000000000000002e-08 pois não estamos vendo melhoria na perda de validação
criado novo otimizador com taxa de aprendizado 1.0000000000000002e-08
sem melhoria em 3 epochs, interrompendo
Treinamento completo em 104m 44s
0
160
320
480
640
800
960
1120
1280
1440
1600
1760
1920
2080
2240
2400
2560
2720
2880
3040
3200
3360
3520
3680
3840
4000
4160
4320
4480
4640
4800
4960
5120
5280
5440
5600
5760
5920
6080
6240
6400
6560
6720
6880
7040
7200
7360
7520
7680
7840
8000
8160
8320
8480
8640
8800
8960
9120
9280
9440
9600
9760
9920
10080
10240
10400
10560
10720
10880
11040
11200
11360
11520
11680
11840
12000
12160
12320
12480
12640
12800
12960
13120
13280
13440
13600
13760
13920
14080
14240
14400
14560
14720
14880
15040
15200
15360
15520
15680
15840
16000
16160
16320
16480
16640
16800
16960
17120
17280
17440
17600
17760
17920
18080
18240
18400
18560
18720
18880
19040
19200
19360
19520
1968